In [1]:
from sklearn.externals import joblib
import gc
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import torch
import numpy as np
import swifter
import torch.optim as optim
import pandas as pd
import time
import random

/root/anaconda3/envs/tianchi_env/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
/root/anaconda3/envs/tianchi_env/lib/python3.6/site-packages/swifter/swifter.py:14: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.errors', please update to use 'numba.core.errors' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.errors import TypingError
/root/anaconda3/envs/tianchi_env/lib/python3.6/site-packages/swifter/swifter.py:14: NumbaDeprecationWarning: An import was requested from a module that has m

# CONFIGURATION

In [2]:
config={
    'lstm_dropout':0.0,
    'char_embedding_size':600,
    'img_pos_size':2048+5+600,
    'query_lstm_hidden_dims':64,
    'img_lstm_hidden_dims':512,
    'rnn_layers':1,
    'conv_in_dim':600,
    'conv_out_dim':128,
    'conv_kernel_size':3,
    'img_vec_dim':2053,
    'model_dim':768,
    'encoder_layers':6,
    'heads':8,
    'ffn_dim':2048,
}

In [3]:
MAX_BOX = 12
MAX_QUERY_CHAR = 14

# MODEL

In [4]:
class Model(nn.Module):
    def __init__(self, config):
        super(Model, self).__init__()
        self.keep_dropout = config['lstm_dropout']
        self.char_embedding_size = config['char_embedding_size']
        self.img_pos_size = config['img_pos_size']
        self.query_lstm_hidden_dims = config['query_lstm_hidden_dims']
        self.img_lstm_hidden_dims = config['img_lstm_hidden_dims']
        
        self.rnn_layers = config['rnn_layers']

        self.conv_in_dim = config['conv_in_dim']
        self.conv_out_dim = config['conv_out_dim']
        self.conv_kernel_size = config['conv_kernel_size']

        self.dense_input_dim = config['img_vec_dim']
        self.dense_output_dim = config['model_dim']

        self.lstm_net_query = nn.LSTM(self.char_embedding_size, self.query_lstm_hidden_dims,
                                      num_layers=self.rnn_layers, dropout=self.keep_dropout,
                                      bidirectional=True, batch_first=True)
    
        self.lstm_net_img = nn.LSTM(self.img_pos_size, self.img_lstm_hidden_dims,
                                      num_layers=self.rnn_layers, dropout=self.keep_dropout,
                                      bidirectional=True, batch_first=True)

        self.mlpLayer = nn.Sequential(nn.BatchNorm1d(768+2*64+1024),nn.ReLU(), nn.Linear(768+2*64+1024, 2048), 
                                      nn.BatchNorm1d(2048),nn.ReLU(), nn.Linear(2048, 512), 
                                      nn.BatchNorm1d(512), nn.ReLU(), nn.Linear(512, 128),
                                      nn.Linear(128, 1))

    def forward(self, query_vec, label_text_vec, query_bert_vec, img_vec, pos_vec,lstm_seq_length,img_lstm_seq_length):
        pack = nn.utils.rnn.pack_padded_sequence(query_vec, lstm_seq_length, batch_first=True,enforce_sorted=False)
        lstm_output, (h, c) = self.lstm_net_query(pack)
        h = h.permute(1, 0, 2)
        h = torch.cat((h[:,0,:],h[:,1,:]),dim=1)  # 2*64=128 dim
        # 768+128=896
        query_bert_vec = torch.cat((query_bert_vec, h.view(query_bert_vec.shape[0], query_bert_vec.shape[1], 2*self.query_lstm_hidden_dims)),dim=-1)
#         print('query_bert_vec size:', query_bert_vec.shape)
        
        img_vec = torch.cat((img_vec, pos_vec), dim=2)
        img_vec = torch.cat((img_vec, label_text_vec), dim=2) # 2048+5+600=2653
        img_pack = nn.utils.rnn.pack_padded_sequence(img_vec, img_lstm_seq_length, batch_first=True,enforce_sorted=False)
        img_lstm_output, (img_h, img_c) = self.lstm_net_img(img_pack)
        img_h = img_h.permute(1,0,2)
        img_h = torch.cat((img_h[:,0,:],img_h[:,1,:]),dim=1) # 2*512=1024 dim
        img_h = img_h.view(query_bert_vec.shape[0], query_bert_vec.shape[1], 2*self.img_lstm_hidden_dims)
        
        output = torch.cat((query_bert_vec, img_h), dim=-1) # 1024+896
        output = self.mlpLayer(output[:,0,:])
        
        return output,torch.cat((query_bert_vec, img_h), dim=-1)[:,0,:]


In [5]:
# loss function
loss_f = nn.BCEWithLogitsLoss()
# sigmoid=nn.Sigmoid()
# def loss_f(predict, label):
#     predict=sigmoid(predict)
#     return (-label*torch.log(predict)-(1-label)*torch.log((1-predict)*0.6)).mean()

def calc_loss(predict, label):
    loss = loss_f(predict.squeeze(dim=-1), label)
    return loss

In [6]:
# generate neg samples
from sklearn.utils import shuffle

# def generate_negative_sample(train):
#     train.reset_index(drop=True, inplace=True)
#     df1 = shuffle(train, random_state=2020)
#     df1.reset_index(drop=True, inplace=True)
#     df1['old_query_id'] = train['query_id']
#     df1['old_product_id'] = train['product_id']
#     df1['query_albert_embedding'] = train['query_albert_embedding']
#     df1['query_embedding'] = train['query_embedding']

#     df1['label'] = df1.apply(lambda x: False if x['query_id']!=x['old_query_id']
#                                                         and x['product_id']!=x['old_product_id'] else True, axis=1)

#     output = df1[df1['label']==False]
#     output['label']=0
#     output = output[train.columns]
#     print('negative sample number:', output.shape[0])
#     del df1
#     gc.collect()
#     return output

def generate_negative_sample(train, num):
    train.reset_index(drop=True, inplace=True)
    output=pd.DataFrame()
    if num<1:
        df = shuffle(train, random_state=2020)
        df.reset_index(drop=True, inplace=True)
        df['old_query_id'] = train['query_id']
        df['old_product_id'] = train['product_id']
        df['query_albert_embedding'] = train['query_albert_embedding']
        df['query_embedding'] = train['query_embedding']
        df['num_query_char'] = train['num_query_char']

        df['label'] = df.apply(lambda x: 0 if x['query_id']!=x['old_query_id']
                                                            and x['product_id']!=x['old_product_id'] else 1, axis=1)
        df = df[train.columns].iloc[0:int(df.shape[0]*num), :]
        output = pd.concat((output, df), axis=0)
        del df
        gc.collect()
    else:
        for i in range(0, num):
            df = shuffle(train, random_state=2020+i)
            df.reset_index(drop=True, inplace=True)
            df['old_query_id'] = train['query_id']
            df['old_product_id'] = train['product_id']
            df['query_albert_embedding'] = train['query_albert_embedding']
            df['query_embedding'] = train['query_embedding']
            df['num_query_char'] = train['num_query_char']

            df['label'] = df.apply(lambda x: 0 if x['query_id']!=x['old_query_id']
                                                                and x['product_id']!=x['old_product_id'] else 1, axis=1)
            df = df[train.columns]
            output = pd.concat((output, df), axis=0)
            del df
            gc.collect()
    
    output.reset_index(drop=True, inplace=True)
    print('negative sample number:', output.shape[0])

    return output

def generate_hard_neg_sample(big_train, train, cat_dict, num):
    train.reset_index(drop=True, inplace=True)
    output=pd.DataFrame()
    for i in range(1, num+1):
        # random without seed 
        train['product_id_new']=train.swifter.apply(lambda x: 
                                                           random.choice(cat_dict[x['random_class_labels'][-i:][0]]), 
                                                           axis=1)

        # random with seed
#         train['product_id_new']=train.swifter.apply(lambda x: 
#                                                            shuffle(cat_dict[x['random_class_labels'][-i:][0]], 
#                                                                    random_state=x['product_id']+i)[-i:][0], 
#                                                            axis=1)
    
        train.sort_values(by='product_id_new', ascending=True, inplace=True)
        train.reset_index(drop=True, inplace=True)
        
        neg_train=big_train.loc[big_train['product_id'].isin(train['product_id_new'])]

        if neg_train.shape[0] < train.shape[0]:
            neg_train=pd.concat((neg_train, neg_train.iloc[0:(train.shape[0]-neg_train.shape[0]), :]), axis=0)
        if neg_train.shape[0] > train.shape[0]:
            neg_train=neg_train.iloc[0:train.shape[0], :]
        neg_train.reset_index(drop=True, inplace=True) 
        
        neg_train['old_query_id'] = train['query_id']
        neg_train['old_product_id'] = train['product_id']
        neg_train['query_albert_embedding'] = train['query_albert_embedding']
        neg_train['query_embedding'] = train['query_embedding']
        neg_train['num_query_char'] = train['num_query_char']

        neg_train['label'] = neg_train.apply(lambda x: 0 if x['query_id']!=x['old_query_id']
                                                            and x['product_id']!=x['old_product_id'] else 1, axis=1)
        neg_train = neg_train[['product_id','query_id','features','label_embedding',
                               'query_embedding','pos','num_box','num_query_char','query_albert_embedding','label']]
        
        output=pd.concat((output, neg_train), axis=0)
        
        del neg_train
        gc.collect()
        
    train.drop(['random_class_labels','product_id_new'],axis=1, inplace=True)
        
    output.reset_index(drop=True, inplace=True)
    print('negative sample number:', output.shape[0])
    return output


# TRAIN DATA

# TRAIN MODEL

In [8]:
import warnings
warnings.filterwarnings("ignore")
# betas = (0.5, 0.999)
lr = 0.0002
weight_decay = 0

model_ft = Model(config).to(torch.device("cuda"))
model_ft.load_state_dict(torch.load('model/2_1200000_bilstm3_b_parameter.pkl'))
params_to_update = list(model_ft.parameters())
# Observe that all parameters are being optimized
optimizer = optim.Adam(params_to_update, lr=lr)
model_ft.train()
print("starting")

starting


In [ ]:
# load 100000 samples 
def load_train_data(train_albert_path, train_data_path):
    train_albert_embedded=joblib.load(train_albert_path)
    train=joblib.load(train_data_path)
    
    train.reset_index(drop=True, inplace=True)
    train_albert_embedded.reset_index(drop=True, inplace=True)
    
    # truncate
    train['label_embedding']=train.swifter.apply(lambda x: x['label_embedding'][:MAX_BOX],axis=1)
    train['pos']=train.swifter.apply(lambda x: x['pos'][:MAX_BOX],axis=1)
    train['query_embedding']=train.swifter.apply(lambda x: x['query_embedding'][:MAX_QUERY_CHAR],axis=1)
    train['num_box']=train.swifter.apply(lambda x: len(x['pos']),axis=1)
    train['num_query_char']=train.swifter.apply(lambda x: len(x['query_embedding']),axis=1)
    train['features']=train.swifter.apply(lambda x: [i for i in x['features']],axis=1)
    train['features']=train.swifter.apply(lambda x: x['features'][:MAX_BOX],axis=1)
    train['query_albert_embedding']=train_albert_embedded[0]
    del train_albert_embedded
    gc.collect()
    return train

def generate_train(train, allsizes):
    length = train.shape[0]
    for i in range(0, length, allsizes):
        yield train.iloc[i:i+allsizes, :]
        
def generate_batch_train(query_vec, 
                         label_text_vec, 
                         query_bert_vec, 
                         img_vec, 
                         pos_vec,  
                         lstm_seq_length,
                         img_lstm_seq_length,
                         label,
                         allsizes, 
                         batch,
                         neg_num=1):
    length = int(allsizes*(1+neg_num))
    for i in range(0, length, batch):
        yield query_vec[i:i+batch], label_text_vec[i:i+batch], query_bert_vec[i:i+batch], img_vec[i:i+batch], pos_vec[i:i+batch], lstm_seq_length[i:i+batch], img_lstm_seq_length[i:i+batch],label[i:i+batch]
        
def cal_prec(predict,labels):
    prec=0
    for p, l in zip(predict,labels):
        if p>0.5 and l==1:
            prec+=1
        if p<=0.5 and l==0:
            prec+=1
            
    return prec/len(predict)

allsizes=2000
batch=1000
epoch=100
neg_num=1

# for only 100000 samples
# hard neg sample--------
# s=1
# train_albert_path='../albert/avgpool_albert_embedded_'+str(s)+'00000'
# train_data_path='data/temp_data_'+str(s)+'00000.pkl'
# train = load_train_data(train_albert_path, train_data_path)

# random_class_labels=joblib.load('../cat/random_class_labels_'+str(s*100000))
# train['random_class_labels']=random_class_labels
# del random_class_labels
# gc.collect()

# print('load '+str(s*100000)+' train samples----------------------')

# cat_dict=joblib.load('../cat/cat_dict_'+str(s*100000))
# train.sort_values(by='product_id', ascending=True, inplace=True)
# train.reset_index(drop=True, inplace=True)
#-------------
tmp=12
for e in range(2, epoch):
    print('epoch:', e)
    if e>2:
        tmp=1
    for s in range(tmp, 31):
#         train_albert_path='../albert/avgpool_albert_embedded_'+str(s)+'00000'
#         train_data_path='data/temp_data_'+str(s)+'00000.pkl'
#         train = load_train_data(train_albert_path, train_data_path)
#         train=shuffle(train, random_state=2020)
#         train.reset_index(drop=True, inplace=True)
#         gc.collect()
#         #-------------

        # hard neg sample--------
        train_albert_path='../albert/avgpool_albert_embedded_'+str(s)+'00000'
        train_data_path='data/temp_data_'+str(s)+'00000.pkl'
        train = load_train_data(train_albert_path, train_data_path)

        random_class_labels=joblib.load('../cat/random_class_labels_'+str(s*100000))
        train['random_class_labels']=random_class_labels
        del random_class_labels
        gc.collect()

        print('load '+str(s*100000)+' train samples----------------------')

        cat_dict=joblib.load('../cat/cat_dict_'+str(s*100000))
        train.sort_values(by='product_id', ascending=True, inplace=True)
        train.reset_index(drop=True, inplace=True)
        #-------------
    
        trained_samples=(s-1)*100000
        for big_batch_train in generate_train(train, allsizes):
            trained_samples += allsizes
            big_batch_train['label']=1

            # padding
            def padding_seq(sequence, length, max_length):
                if length >= max_length:
                    return sequence
                else:
                    return sequence + [np.zeros(len(sequence[0])) for _ in range(max_length-length)]

            # generate negative samples
#             negative_train = generate_negative_sample(big_batch_train,neg_num)
            # generate hard negative samples
            negative_train = generate_hard_neg_sample(train, big_batch_train, cat_dict, neg_num)         
        
            big_batch_train = pd.concat((big_batch_train, negative_train), axis=0)
            big_batch_train['label_embedding']=big_batch_train.swifter.apply(lambda x: padding_seq(x['label_embedding'], x['num_box'], MAX_BOX),axis=1)
            big_batch_train['pos']=big_batch_train.swifter.apply(lambda x: padding_seq(x['pos'], x['num_box'], MAX_BOX),axis=1)
            big_batch_train['query_embedding']=big_batch_train.swifter.apply(lambda x: padding_seq(x['query_embedding'], x['num_query_char'], MAX_QUERY_CHAR),axis=1)
            big_batch_train['features']=big_batch_train.swifter.apply(lambda x: padding_seq(x['features'], x['num_box'], MAX_BOX),axis=1)
            gc.collect()
    
            big_batch_train.reset_index(drop=True, inplace=True)
            big_batch_train=shuffle(big_batch_train, random_state=2020)
            big_batch_train.reset_index(drop=True, inplace=True)


            query_vec = torch.Tensor(big_batch_train['query_embedding'])
            lstm_seq_length = list(big_batch_train['num_query_char'])
            img_lstm_seq_length = list(big_batch_train['num_box'])

            label_text_vec = torch.Tensor(big_batch_train['label_embedding'])
            query_bert_vec = torch.Tensor(big_batch_train['query_albert_embedding']).view(int(allsizes*(1+neg_num)), 1, 768)

            img_vec = torch.Tensor(big_batch_train['features'])
            pos_vec = torch.Tensor(big_batch_train['pos'])

            label =torch.Tensor(big_batch_train['label'])

            del big_batch_train, negative_train
            gc.collect()

            # train model in per batch samples
            train_count = 0
            avg_loss = 0
            avg_prec = 0
            for i in generate_batch_train(query_vec,label_text_vec,query_bert_vec,img_vec,pos_vec,lstm_seq_length,img_lstm_seq_length,label,allsizes, batch, neg_num):
                b_query_vec, b_label_text_vec, b_query_bert_vec, b_img_vec, b_pos_vec, b_lstm_seq_length, b_img_lstm_seq_length, b_label=i
                optimizer.zero_grad()
                prediction=model_ft(b_query_vec.cuda(), b_label_text_vec.cuda(), b_query_bert_vec.cuda(), b_img_vec.cuda(), 
                                    b_pos_vec.cuda(), b_lstm_seq_length, b_img_lstm_seq_length)

                loss = calc_loss(prediction, b_label.cuda())
                prec = cal_prec(prediction, b_label.cuda())

                avg_loss += loss.item()
                avg_prec += prec
                loss.backward()
                optimizer.step()
                train_count += 1

            print('epoch:', e, 'trained samples:', trained_samples, '----average loss:', avg_loss / train_count)
            print('epoch:', e, 'trained samples:', trained_samples, '----average prec:', avg_prec / train_count)

            del query_vec,lstm_seq_length,img_lstm_seq_length,label_text_vec,query_bert_vec,img_vec,pos_vec
            gc.collect()
            if trained_samples % 20000 == 0:
                torch.save(model_ft.state_dict(), 'model/' + str(e) + '_'+ str(trained_samples) + '_bilstm3_b_parameter.pkl')
                print('model/' + str(e) + '_' + str(trained_samples) + '_bilstm3_b_parameter.pkl has been saved!')
            
        del train
        gc.collect()
        
#         torch.save(model_ft.state_dict(), 'model/' + str(e) + '_'+ str(s*100000) + '_bilstm3_b_parameter.pkl')
#         print('model/' + str(e) + '_' + str(s*100000) + '_bilstm3_b_parameter.pkl has been saved!')


epoch: 2



load 1200000 train samples----------------------



negative sample number: 2000



epoch: 2 trained samples: 1102000 ----average loss: 0.1768045835196972
epoch: 2 trained samples: 1102000 ----average prec: 0.9325000000000001



negative sample number: 2000


...


epoch: 3 trained samples: 1092000 ----average loss: 0.17501351982355118
epoch: 3 trained samples: 1092000 ----average prec: 0.9272500000000001



negative sample number: 2000


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




load 1200000 train samples----------------------



negative sample number: 2000


In [32]:
print(query_vec[0:10].shape, 
      label_text_vec[0:10].shape, 
      query_bert_vec[0:10].shape, 
      img_vec[0:10].shape, 
      pos_vec[0:10].shape, 
      np.array(lstm_seq_length[0:10]).shape,
      np.array(img_lstm_seq_length[0:10]).shape,
     )

for i in range(100):
    optimizer.zero_grad()
    prediction=model_ft(b_query_vec.cuda(), b_label_text_vec.cuda(), b_query_bert_vec.cuda(), b_img_vec.cuda(), 
                                    b_pos_vec.cuda(), b_lstm_seq_length, b_img_lstm_seq_length)
    loss = calc_loss(prediction, labels.cuda())
    print(str(i)+':loss-------',loss)

#             print('train prec:', cal_prec(predict,labels))

    loss.backward()
    optimizer.step()

torch.Size([10, 12, 600]) torch.Size([10, 10, 600]) torch.Size([10, 1, 768]) torch.Size([10, 10, 2048]) torch.Size([10, 10, 5]) (10,) (10,)
output size: torch.Size([250, 1, 768])


RuntimeError: running_mean should contain 1 elements not 768

# VALID DATA

In [7]:
# load data
valid_albert_embedding=joblib.load('../albert/avgpool_albert_embedded_valid')
valid=joblib.load('data/valid_data.pkl')
# truncate
valid['label_embedding']=valid.swifter.apply(lambda x: x['label_embedding'][:MAX_BOX],axis=1)
valid['pos']=valid.swifter.apply(lambda x: x['pos'][:MAX_BOX],axis=1)
valid['query_embedding']=valid.swifter.apply(lambda x: x['query_embedding'][:MAX_QUERY_CHAR],axis=1)
valid['num_box']=valid.swifter.apply(lambda x: len(x['pos']),axis=1)
valid['num_query_char']=valid.swifter.apply(lambda x: len(x['query_embedding']),axis=1)
valid['features']=valid.swifter.apply(lambda x: [i for i in x['features']],axis=1)
valid['features']=valid.swifter.apply(lambda x: x['features'][:MAX_BOX],axis=1)
# padding
def padding_seq(sequence, length, max_length):
    if length >= max_length:
        return sequence
    else:
        return sequence + [np.zeros(len(sequence[0])) for _ in range(max_length-length)]
    
valid['label_embedding']=valid.swifter.apply(lambda x: padding_seq(x['label_embedding'], x['num_box'], MAX_BOX),axis=1)
valid['pos']=valid.swifter.apply(lambda x: padding_seq(x['pos'], x['num_box'], MAX_BOX),axis=1)
valid['query_embedding']=valid.swifter.apply(lambda x: padding_seq(x['query_embedding'], x['num_query_char'], MAX_QUERY_CHAR),axis=1)
valid['features']=valid.swifter.apply(lambda x: padding_seq(x['features'], x['num_box'], MAX_BOX),axis=1)
valid['query_albert_embedding']=valid_albert_embedding[0]
gc.collect()
valid

,product_id,query_id,features,label_embedding,query_embedding,pos,num_box,num_query_char,query_albert_embedding
0,103047314,0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.04083252, -0.043029785, 0.17004395, -0.861...","[[-0.18566895, 0.065979004, -0.25219727, -0.11...","[[0.105, 0.4175, 0.1425, 0.59625, 0.1417966534...",2,3,"[-0.0056863907, -0.63583577, -0.42185646, -0.5..."
1,103030678,0,"[[0.0, 0.0, 0.0022290119, 0.0, 0.0, 0.18552563...","[[0.04083252, -0.043029785, 0.17004395, -0.861...","[[-0.18566895, 0.065979004, -0.25219727, -0.11...","[[0.52375, 0.97375, 0.00625, 0.29125, 0.128249...",9,3,"[-0.0056863907, -0.63583577, -0.42185646, -0.5..."
2,103031217,0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.04083252, -0.043029785, 0.17004395, -0.861...","[[-0.18566895, 0.065979004, -0.25219727, -0.11...","[[0.264, 0.564, 0.09221311475409837, 0.9672131...",2,3,"[-0.0056863907, -0.63583577, -0.42185646, -0.5..."
3,103026621,0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-0.27514648, 0.08642578, -0.28344727, -0.252...","[[-0.18566895, 0.065979004, -0.25219727, -0.11...","[[0.4325, 0.63125, 0.566200215285253, 0.725511...",10,3,"[-0.0056863907, -0.63583577, -0.42185646, -0.5..."
4,103017048,0,"[[0.0, 0.0, 0.05747965, 0.0, 0.0, 0.0, 0.0, 0....","[[0.04083252, -0.043029785, 0.17004395, -0.861...","[[-0.18566895, 0.065979004, -0.25219727, -0.11...","[[0.2375, 0.6975, 0.34875, 0.90375, 0.25529960...",2,3,"[-0.0056863907, -0.63583577, -0.42185646, -0.5..."
...,...,...,...,...,...,...,...,...,...
14715,103044330,500,"[[0.0, 0.0, 0.027551055, 0.0, 0.32401124, 0.0,...","[[-0.19830322, 0.09790802, 0.42492676, -0.3308...","[[-0.59521484, 0.828125, -0.07165527, -0.15124...","[[0.24375, 0.41625, 0.04875, 0.23875, 0.032774...",5,3,"[0.01230786, -0.51479, -0.36992726, -0.6571593..."
14716,103016765,500,"[[0.0, 0.0, 4.595919, 0.0, 0.0, 0.03517687, 0....","[[0.107177734, -0.19818115, -0.11024475, 0.392...","[[-0.59521484, 0.828125, -0.07165527, -0.15124...","[[0.16533333333333333, 1.3266666666666667, 0.1...",3,3,"[0.01230786, -0.51479, -0.36992726, -0.6571593..."
14717,103013406,500,"[[0.017800761, 0.0, 0.050141126, 0.0, 0.0, 0.0...","[[0.10523733, -0.16297404, -0.12767537, -0.051...","[[-0.59521484, 0.828125, -0.07165527, -0.15124...","[[0.28, 1.4893333333333334, 0.0, 0.66666666666...",3,3,"[0.01230786, -0.51479, -0.36992726, -0.6571593..."
14718,103012098,500,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-0.27514648, 0.08642578, -0.28344727, -0.252...","[[-0.59521484, 0.828125, -0.07165527, -0.15124...","[[0.135, 0.211, 0.888, 0.996, 0.00820799179200...",6,3,"[0.01230786, -0.51479, -0.36992726, -0.6571593..."


In [17]:
# predict

import warnings
warnings.filterwarnings("ignore")
import json
import sys
import os
import string
import numpy as np
import time
import glob
import zipfile
import shutil


def glob_matching(fn, fmt):
    matched_fns = list(glob.iglob('submit/**/submission.csv', recursive=True))
    if len(matched_fns) == 0:
        raise Exception("You submitted a {} file, but we didn't find submission.csv in it. Please check your submission.".format(fmt))
    if len(matched_fns) > 1:
        raise Exception("You submitted a {} file, but there are more than one files named submission.csv in it. Please check your submission.".format(fmt))
    return matched_fns[0]


def read_submission(submit_path, reference, k=5):

    submission_dict = {}
    ref_qids = set(reference.keys())
    
    with open(submit_path) as fin:
        for line in fin:
            line = line.strip()
            records = [elem.strip() for elem in line.split(',')]
            if records[0] not in ref_qids:
                continue
            qid = records[0]
            # check whether there are K products for each query
            if len(records[1:]) != k:
                raise Exception('Query-id {} has wrong number of predicted product-ids! Require {}, but {} founded.'.format(qid, k, len(records[1:])))
            # check whether there exists an empty prediction for any query
            if any([len(r) == 0 for r in records[1:]]):
                raise Exception('Query-id {} has an empty prediction at rank {}! Pleace check again!'.format(qid, records[1:].index("") + 1))            
            # check whether there exist an invalid prediction for any query
            for rank, r in enumerate(records[1:]):
                if not all([char in string.digits for char in r]):
                    raise Exception('Query-id {} has an invalid prediction product-id \"{}\" at rank {}'.format(qid, r, rank + 1))
            # check whether there are duplicate predicted products for a single query
            if len(set(records[1:])) != k:
                raise Exception('Query-id {} has duplicate products in your prediction. Pleace check again!'.format(qid))
            submission_dict[qid] = records[1:] # here we save the list of string
    
    # check if any query is missing in the submission
    pred_qids = set(submission_dict.keys())
    nopred_qids = ref_qids - pred_qids
    if len(nopred_qids) != 0:
        raise Exception('The following query-ids have no prediction in your submission, please check again: {}'.format(", ".join(nopred_qids)))

    return submission_dict


# compute dcg@k for a single sample
def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    if r.size:
        return r[0] + np.sum(r[1:] / np.log2(np.arange(3, r.size + 2)))
    return 0.


# compute ndcg@k (dcg@k / idcg@k) for a single sample
def get_ndcg(r, ref, k):
    dcg_max = dcg_at_k(ref, k)
    if not dcg_max:
        return 0.
    dcg = dcg_at_k(r, k)
    return dcg / dcg_max


def valid_evaluate(standard_path, submit_path):
    # the path of answer json file (eg. valid_answer.json)
    # the path of prediction file (csv or zip)
    try:
        # read ground-truth
        reference = json.load(open(standard_path))
        
        # read predictions
        k = 5
        predictions = read_submission(submit_path, reference, k)

        # compute score for each query
        score_sum = 0.
        for qid in reference.keys():
            ground_truth_ids = set([str(pid) for pid in reference[qid]])
            ref_vec = [1.0] * len(ground_truth_ids)
            pred_vec = [1.0 if pid in ground_truth_ids else 0.0 for pid in predictions[qid]]
            score_sum += get_ndcg(pred_vec, ref_vec, k)
        # the higher score, the better
        score = score_sum / len(reference)
        return score
    except Exception as e:
        print("The evaluation failed.")
        raise e

t=212
for model_i in range(10, 12):
    if model_i>10:
        t=2
    for j in range(t, 302, 2):
        model_ft = Model(config).to(torch.device("cpu"))
        model_ft.load_state_dict(torch.load('model/'+str(model_i)+'_'+str(j)+'0000_bilstm3_b_parameter.pkl'))
        model_ft.eval()

        query_vec = torch.Tensor(valid['query_embedding'])
        lstm_seq_length = list(valid['num_query_char'])
        img_lstm_seq_length = list(valid['num_box'])
        label_text_vec = torch.Tensor(valid['label_embedding'])
        query_bert_vec = torch.Tensor(valid['query_albert_embedding']).view(14720, 1, 768)
        img_vec = torch.Tensor(valid['features'])
        pos_vec = torch.Tensor(valid['pos'])

        gc.collect() 

        # prediction=model_ft(query_vec.cuda(), label_text_vec.cuda(), query_bert_vec.cuda(), img_vec.cuda(), 
        #                     pos_vec.cuda(), padding_mask.cuda(), lstm_seq_length)
        res = []
        for i in range(0, 14720, 1000):
            prediction=model_ft(query_vec[i:i+1000], label_text_vec[i:i+1000], query_bert_vec[i:i+1000], img_vec[i:i+1000], 
                                pos_vec[i:i+1000], lstm_seq_length[i:i+1000], img_lstm_seq_length[i:i+1000])
            res += [i[0].item() for i in prediction.detach().numpy()]
            gc.collect()

        valid['prediction']=res
        sorted_valid=valid.sort_values(by=['query_id','prediction'],ascending=[True, False])
        submission=[]
        for i in valid['query_id'].unique():
            submission.append([i]+list(sorted_valid[sorted_valid['query_id']==i]['product_id'].iloc[:5]))
        submission=pd.DataFrame(submission, columns=['query-id','product1','product2',
                                                     'product3','product4','product5'],dtype=np.int)

        submission.to_csv('valid_pred.csv', index=0)

        standard_path= '../../tianchi_data/valid_answer.json'
        submit_path='valid_pred.csv'
        score = valid_evaluate(standard_path, submit_path)
        print(model_i, j, "score:", score)
        
#         time.sleep(3*60)


# 7  180 0.620
# 8  88  0.620   20_1
# 8  132 0.620   all
# 9  268 0.624
# 9  272 0.626
# 11 80  0.623
# 11 108 0.624

10 212 score: 0.6118105749019231
10 214 score: 0.5954343039714302
10 216 score: 0.6073918072577627
10 218 score: 0.6169375216875944
10 220 score: 0.6169761716776074
10 222 score: 0.60762728333367
10 224 score: 0.5973398373829467
10 226 score: 0.591531966333725
10 228 score: 0.610315011451869
10 230 score: 0.5963763999331554
10 232 score: 0.5865517077524508
10 234 score: 0.5937766176083953
10 236 score: 0.5984659142626826
10 238 score: 0.588854476781752
10 240 score: 0.598028947848919
10 242 score: 0.6117989470285622
10 244 score: 0.6095392132907245
10 246 score: 0.6063404124620219
10 248 score: 0.590031286548264
10 250 score: 0.5915970253843053
10 252 score: 0.5976683127705089
10 254 score: 0.6039543127719625
10 256 score: 0.6140886022972251
10 258 score: 0.6011725620878823
10 260 score: 0.6013243226601472
10 262 score: 0.6024181472395976
10 264 score: 0.5915839060225933
10 266 score: 0.5976207982599663
10 268 score: 0.5993223473799605
10 270 score: 0.5964318405157002
10 272 score: 0.5

FileNotFoundError: [Errno 2] No such file or directory: 'model/11_1160000_bilstm3_b_parameter.pkl'

In [70]:
model_ft = Model(config).to(torch.device("cpu"))
model_ft.load_state_dict(torch.load('model/'+'8_1980000_bilstm3_b_parameter.pkl'))
model_ft.eval()

query_vec = torch.Tensor(valid['query_embedding'])
lstm_seq_length = list(valid['num_query_char'])
img_lstm_seq_length = list(valid['num_box'])
label_text_vec = torch.Tensor(valid['label_embedding'])
query_bert_vec = torch.Tensor(valid['query_albert_embedding']).view(14720, 1, 768)
img_vec = torch.Tensor(valid['features'])
pos_vec = torch.Tensor(valid['pos'])

gc.collect() 

# prediction=model_ft(query_vec.cuda(), label_text_vec.cuda(), query_bert_vec.cuda(), img_vec.cuda(), 
#                     pos_vec.cuda(), padding_mask.cuda(), lstm_seq_length)
res = []
for i in range(0, 14720, 1000):
    prediction=model_ft(query_vec[i:i+1000], label_text_vec[i:i+1000], query_bert_vec[i:i+1000], img_vec[i:i+1000], 
                        pos_vec[i:i+1000], lstm_seq_length[i:i+1000], img_lstm_seq_length[i:i+1000])
    res += [i[0].item() for i in prediction.detach().numpy()]
    gc.collect()

# valid['prediction']=res
# sorted_valid=valid.sort_values(by=['query_id','prediction'],ascending=[True, False])
# submission=[]
# for i in valid['query_id'].unique():
#     submission.append([i]+list(sorted_valid[sorted_valid['query_id']==i]['product_id'].iloc[:5]))
# submission=pd.DataFrame(submission, columns=['query-id','product1','product2',
#                                              'product3','product4','product5'],dtype=np.int)

# submission.to_csv('valid_pred.csv', index=0)

# standard_path= '../../tianchi_data/valid_answer.json'
# submit_path='valid_pred.csv'
# score = valid_evaluate(standard_path, submit_path)
# print(model_i, j, "score:", score)
        


In [71]:
res=[torch.sigmoid(torch.FloatTensor([i])).item() for i in res]
valid['prediction']=res
valid=valid[['product_id', 'query_id', 'prediction']]
valid.to_csv('valid_pred_all_619_1.csv', index=0)

In [74]:
valid['prediction1']=res
valid=valid[['product_id', 'query_id', 'prediction1']]

In [75]:
valid

,product_id,query_id,prediction1
0,103047314,0,0.216208
1,103030678,0,0.001461
2,103031217,0,0.988909
3,103026621,0,0.000003
4,103017048,0,0.995905
...,...,...,...
14715,103044330,500,0.949931
14716,103016765,500,0.004777
14717,103013406,500,0.463796
14718,103012098,500,0.027307


In [25]:
valid_63 = joblib.load('eval_resultscore_0.63')

In [28]:
valid_63[0]

[(103047314, -3.994809627532959),
 (103030678, -6.863636493682861),
 (103031217, 3.1053380966186523),
 (103026621, -9.876502990722656),
 (103017048, 3.664736747741699),
 (103001388, -5.596713066101074),
 (103057842, -5.224282264709473),
 (103020893, 3.3483307361602783),
 (103061530, -5.6033525466918945),
 (103028493, -5.305731773376465),
 (103025465, -6.496056079864502),
 (103027983, 0.38288167119026184),
 (103020398, 4.390899181365967),
 (103026519, -3.2312302589416504),
 (103046705, -7.228700637817383),
 (103051756, 0.24450881779193878),
 (103030528, -3.575307846069336),
 (103020299, -2.8843884468078613),
 (103023399, 3.022508382797241),
 (103035435, -6.740358352661133),
 (103005587, -3.48378324508667),
 (103054747, -6.882498264312744),
 (103024664, -3.596745014190674),
 (103056681, -2.853959083557129),
 (103057125, 1.1057202816009521),
 (103065485, -3.6773018836975098),
 (103007235, -4.072139739990234),
 (103018696, -5.255303382873535),
 (103059361, 4.443154811859131),
 (103012102, 

In [31]:
valid_63_df=pd.DataFrame()
for k in valid_63.keys():
    df=pd.DataFrame()
    df['product_id']=[i[0] for i in valid_63[k]]
    df['query_id']=k
    df['prediction']=[i[1] for i in valid_63[k]]
    valid_63_df=pd.concat((valid_63_df, df), axis=0)
    
valid_63_df.reset_index(drop=True, inplace=True)

In [33]:
valid_63_df['prediction']=valid_63_df['prediction'].apply(lambda x: torch.sigmoid(torch.FloatTensor([x])).item())

In [34]:
valid_63_df

,product_id,query_id,prediction
0,103047314,0,1.807812e-02
1,103030678,0,1.044015e-03
2,103031217,0,9.571124e-01
3,103026621,0,5.136496e-05
4,103017048,0,9.750286e-01
...,...,...,...
14715,103044330,500,8.856184e-01
14716,103016765,500,1.981666e-06
14717,103013406,500,1.882842e-04
14718,103012098,500,3.127934e-06


In [72]:
valid['prediction']= 0.5*valid['prediction']+0.5*valid_63_df['prediction']

In [73]:
sorted_valid=valid.sort_values(by=['query_id','prediction'],ascending=[True, False])
submission=[]
for i in valid['query_id'].unique():
    submission.append([i]+list(sorted_valid[sorted_valid['query_id']==i]['product_id'].iloc[:5]))
submission=pd.DataFrame(submission, columns=['query-id','product1','product2',
                                             'product3','product4','product5'],dtype=np.int)

submission.to_csv('valid_pred.csv', index=0)

standard_path= '../../tianchi_data/valid_answer.json'
submit_path='valid_pred.csv'
score = valid_evaluate(standard_path, submit_path)
print(model_i, j, "score:", score)

9 128 score: 0.6372749037284229


## -------- regarded as baseline ----------

# TEST DATA

In [17]:
test_albert_embedding=joblib.load('../albert/avgpool_albert_embedded_testB')
test=joblib.load('data/testB_data.pkl')
# truncate
# test['label_embedding']=test.swifter.apply(lambda x: x['label_embedding'][:MAX_BOX],axis=1)
# test['pos']=test.swifter.apply(lambda x: x['pos'][:MAX_BOX],axis=1)
# test['query_embedding']=test.swifter.apply(lambda x: x['query_embedding'][:MAX_QUERY_CHAR],axis=1)
# test['num_box']=test.swifter.apply(lambda x: len(x['pos']),axis=1)
# test['num_query_char']=test.swifter.apply(lambda x: len(x['query_embedding']),axis=1)
# test['features']=test.swifter.apply(lambda x: [i for i in x['features']],axis=1)
# test['features']=test.swifter.apply(lambda x: x['features'][:MAX_BOX],axis=1)
# # padding
# def padding_seq(sequence, length, max_length):
#     if length >= max_length:
#         return sequence
#     else:
#         return sequence + [np.zeros(len(sequence[0])) for _ in range(max_length-length)]
    
# test['label_embedding']=test.swifter.apply(lambda x: padding_seq(x['label_embedding'], x['num_box'], MAX_BOX),axis=1)
# test['pos']=test.swifter.apply(lambda x: padding_seq(x['pos'], x['num_box'], MAX_BOX),axis=1)
# test['query_embedding']=test.swifter.apply(lambda x: padding_seq(x['query_embedding'], x['num_query_char'], MAX_QUERY_CHAR),axis=1)
# test['features']=test.swifter.apply(lambda x: padding_seq(x['features'], x['num_box'], MAX_BOX),axis=1)
# test['query_albert_embedding']=test_albert_embedding[0]
# gc.collect()

# truncate
test['label_embedding']=test.swifter.apply(lambda x: x['label_embedding'][:MAX_BOX],axis=1)
test['pos']=test.swifter.apply(lambda x: x['pos'][:MAX_BOX],axis=1)
test['query_embedding']=test.swifter.apply(lambda x: x['query_embedding'][:MAX_QUERY_CHAR],axis=1)
test['num_box']=test.swifter.apply(lambda x: len(x['pos']),axis=1)
test['num_query_char']=test.swifter.apply(lambda x: len(x['query_embedding']),axis=1)
test['features']=test.swifter.apply(lambda x: [i for i in x['features']],axis=1)
test['features']=test.swifter.apply(lambda x: x['features'][:MAX_BOX],axis=1)
# padding
def padding_seq(sequence, length, max_length):
    if length >= max_length:
        return sequence
    else:
        return sequence + [np.zeros(len(sequence[0])) for _ in range(max_length-length)]
    
test['label_embedding']=test.swifter.apply(lambda x: padding_seq(x['label_embedding'], x['num_box'], MAX_BOX),axis=1)
test['pos']=test.swifter.apply(lambda x: padding_seq(x['pos'], x['num_box'], MAX_BOX),axis=1)
test['query_embedding']=test.swifter.apply(lambda x: padding_seq(x['query_embedding'], x['num_query_char'], MAX_QUERY_CHAR),axis=1)
test['features']=test.swifter.apply(lambda x: padding_seq(x['features'], x['num_box'], MAX_BOX),axis=1)
test['query_albert_embedding']=test_albert_embedding[0]
gc.collect()
test

,product_id,query_id,features,label_embedding,query_embedding,pos,num_box,num_query_char,query_albert_embedding
0,103042011,0,"[[0.0, 0.026633915, 0.259833, 0.0, 0.23220193,...","[[-0.27514648, 0.08642578, -0.28344727, -0.252...","[[0.37768555, -0.29248047, -0.48217773, -0.097...","[[0.13875, 0.3425, 0.535, 0.67625, 0.028779642...",9,3,"[-0.31451425, -0.08342799, -0.03945126, -0.905..."
1,101177314,0,"[[0.0, 0.0, 6.9271755, 0.0, 0.43702954, 0.0, 0...","[[-0.27514648, 0.08642578, -0.28344727, -0.252...","[[0.37768555, -0.29248047, -0.48217773, -0.097...","[[0.32666666666666666, 0.44666666666666666, 0....",4,3,"[-0.31451425, -0.08342799, -0.03945126, -0.905..."
2,103009949,0,"[[0.0, 0.0, 0.013801694, 0.0, 0.011535197, 0.0...","[[-0.27514648, 0.08642578, -0.28344727, -0.252...","[[0.37768555, -0.29248047, -0.48217773, -0.097...","[[0.1725, 0.88625, 0.6525, 0.9775, 0.231968387...",6,3,"[-0.31451425, -0.08342799, -0.03945126, -0.905..."
3,103022608,0,"[[0.0, 0.12027071, 0.9106935, 0.0, 0.61674774,...","[[-0.27514648, 0.08642578, -0.28344727, -0.252...","[[0.37768555, -0.29248047, -0.48217773, -0.097...","[[0.52875, 0.6325, 0.575, 0.66625, 0.009467172...",7,3,"[-0.31451425, -0.08342799, -0.03945126, -0.905..."
4,103041034,0,"[[0.0, 0.055186935, 0.0008050593, 0.0, 0.06662...","[[-0.27514648, 0.08642578, -0.28344727, -0.252...","[[0.37768555, -0.29248047, -0.48217773, -0.097...","[[0.16625, 0.43375, 0.18, 0.6675, 0.1304060462...",5,3,"[-0.31451425, -0.08342799, -0.03945126, -0.905..."
...,...,...,...,...,...,...,...,...,...
29000,103032444,999,"[[0.0, 0.0, 1.0701001, 0.0, 0.003615479, 0.0, ...","[[0.10523733, -0.16297404, -0.12767537, -0.051...","[[-0.07965088, 0.07574463, -0.068725586, 0.384...","[[0.179, 0.68, 0.052, 0.432, 0.190379809620190...",10,2,"[-0.059709996, -0.22220874, -0.2568321, 0.0440..."
29001,103018483,999,"[[0.0, 0.0, 1.1396561, 0.0, 0.0, 0.28721818, 0...","[[-0.45617676, -0.29260254, -0.21557617, 0.159...","[[-0.07965088, 0.07574463, -0.068725586, 0.384...","[[0.192, 0.365, 0.246, 0.35, 0.017991982008017...",2,2,"[-0.059709996, -0.22220874, -0.2568321, 0.0440..."
29002,103011147,999,"[[0.0, 0.0, 0.38087285, 0.0, 0.0, 0.0, 0.0, 0....","[[0.10523733, -0.16297404, -0.12767537, -0.051...","[[-0.07965088, 0.07574463, -0.068725586, 0.384...","[[0.12, 0.8675, 0.06, 0.915, 0.639111501388279...",1,2,"[-0.059709996, -0.22220874, -0.2568321, 0.0440..."
29003,103031908,999,"[[0.0, 0.0, 0.01731833, 0.0, 0.0, 0.51081115, ...","[[0.10523733, -0.16297404, -0.12767537, -0.051...","[[-0.07965088, 0.07574463, -0.068725586, 0.384...","[[0.0, 0.83, 0.0, 1.0, 0.8299987031270264], [0...",1,2,"[-0.059709996, -0.22220874, -0.2568321, 0.0440..."


In [18]:
import warnings
warnings.filterwarnings("ignore")

# 8 88  0.620   20_1
# 8 132 0.620   all
# 8 198 0.619   19_1

model_ft = Model(config).to(torch.device("cpu"))
model_ft.load_state_dict(torch.load('model/8_1980000_bilstm3_b_parameter.pkl'))
model_ft.eval()

query_vec = torch.Tensor(test['query_embedding'])
lstm_seq_length = list(test['num_query_char'])
img_lstm_seq_length = list(test['num_box'])
label_text_vec = torch.Tensor(test['label_embedding'])
query_bert_vec = torch.Tensor(test['query_albert_embedding']).view(29005, 1, 768)
img_vec = torch.Tensor(test['features'])
pos_vec = torch.Tensor(test['pos'])

gc.collect() 

# prediction=model_ft(query_vec.cuda(), label_text_vec.cuda(), query_bert_vec.cuda(), img_vec.cuda(), 
#                     pos_vec.cuda(), padding_mask.cuda(), lstm_seq_length)
res = []
for i in range(0, 29005, 1000):
    prediction=model_ft(query_vec[i:i+1000], label_text_vec[i:i+1000], query_bert_vec[i:i+1000], img_vec[i:i+1000], 
                        pos_vec[i:i+1000], lstm_seq_length[i:i+1000], img_lstm_seq_length[i:i+1000])
    res += [i[0].item() for i in prediction.detach().numpy()]
    gc.collect()

res=[torch.sigmoid(torch.FloatTensor([i])).item() for i in res]
test['prediction']=res
test=test[['product_id', 'query_id', 'prediction']]
test.to_csv('test_pred_all_619_1.csv', index=0)
        
# prediction=model_ft(query_vec, label_text_vec, query_bert_vec, img_vec, 
#                     pos_vec, padding_mask, lstm_seq_length)

# test['prediction']=[i[0].item() for i in prediction.detach().numpy()]
# sorted_test=test.sort_values(by=['query_id','prediction'],ascending=[True, False])
# submission=[]
# for i in test['query_id'].unique():
#     submission.append([i]+list(sorted_test[sorted_test['query_id']==i]['product_id'].iloc[:5]))
# submission=pd.DataFrame(submission, columns=['query-id','product1','product2',
#                                              'product3','product4','product5'],dtype=np.int)

# submission
# submission.to_csv('submission/single_tower_attention_bilstm_8_1000000_05220926_sub.csv', index=0)


In [14]:
test

,product_id,query_id,prediction
0,103042011,0,0.997921
1,101177314,0,0.998446
2,103009949,0,0.925310
3,103022608,0,0.858126
4,103041034,0,0.999475
...,...,...,...
29000,103032444,999,0.447068
29001,103018483,999,0.886764
29002,103011147,999,0.403321
29003,103031908,999,0.054182


In [34]:
label_text_vec=torch.rand(5, 10, 2048)
conv1d = nn.Conv1d(in_channels=2048, out_channels=64,kernel_size=3)
label_text_vec = label_text_vec.permute(0, 2, 1)
conv_output = conv1d(label_text_vec)
print(conv_output.shape)
conv_output = F.max_pool1d(conv_output, kernel_size=3, stride=1)
print(conv_output.shape)
conv_output = conv_output.view(conv_output.shape[0], 1, conv_output.shape[1]*conv_output.shape[2])
print(conv_output.shape)

torch.Size([5, 64, 8])
torch.Size([5, 64, 6])
torch.Size([5, 1, 384])


In [19]:
list(range(2, 10,2))

[2, 4, 6, 8]

In [8]:
for j in range(2, 302, 2):
    print(j)

2
4
6
8
10
12
14
16
18
20
22
24
26
28
30
32
34
36
38
40
42
44
46
48
50
52
54
56
58
60
62
64
66
68
70
72
74
76
78
80
82
84
86
88
90
92
94
96
98
100
102
104
106
108
110
112
114
116
118
120
122
124
126
128
130
132
134
136
138
140
142
144
146
148
150
152
154
156
158
160
162
164
166
168
170
172
174
176
178
180
182
184
186
188
190
192
194
196
198
200
202
204
206
208
210
212
214
216
218
220
222
224
226
228
230
232
234
236
238
240
242
244
246
248
250
252
254
256
258
260
262
264
266
268
270
272
274
276
278
280
282
284
286
288
290
292
294
296
298
300


In [8]:
for i in range(0, 1.05, 0.1):
    print(i)

TypeError: 'float' object cannot be interpreted as an integer

In [16]:
! nvidia-smi

Thu Jun 18 05:47:41 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.129      Driver Version: 410.129      CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla M40           Off  | 00000000:00:08.0 Off |                  Off |
| N/A   35C    P0    60W / 250W |    509MiB / 12215MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla M40           Off  | 00000000:00:09.0 Off |                  Off |
| N/A   29C    P8    17W / 250W |     11MiB / 12215MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        